In [1]:
import pandas as pd
import numpy as np
import re
from nltk.corpus import stopwords
import json
from nltk.tokenize import  word_tokenize
import tensorflow as tf
import ast

/home/pratik/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
# Import word embeddings and reformat
import gensim
from gensim.models.keyedvectors import KeyedVectors
word_vectors = gensim.models.Word2Vec.load('nearby_embeddings2')
wordVectors = []
for word in word_vectors.wv.index2word:
    wordVectors.append(word_vectors.wv[word])
wordVectors = np.array(wordVectors)
wordsList = word_vectors.wv.index2word

max_sequence_length = 20

In [3]:
raw_data = pd.read_csv('/home/pratik/NER/data/large_positive.csv')
len(raw_data)

21682

In [4]:
raw_data.drop_duplicates(subset=['body'], inplace=True)
len(raw_data)

16623

In [5]:
def convert_to_list(original_text_list):
    l = ast.literal_eval(original_text_list)
    l = [i.strip() for i in l]
    return l
raw_data['original_text'] = raw_data['original_text'].apply(lambda x: convert_to_list(x))

In [6]:
def generate_y(text, original_text_list, k):
    y = np.zeros([max_sequence_length, 2])
    for i in range(len(y)):
        y[i][0] = 1
    for original_text_words in original_text_list:
        for original_text in word_tokenize(original_text_words):
            words = word_tokenize(text)
            for i in range(len(words)):
                if words[i].lower() == original_text.lower():
                  #  print(words[i], original_text)
                    y[i][1] = 1
                    y[i][0] = 0
    return y


In [7]:
def generate_y(text, original_text_list, k):
    y = np.zeros([max_sequence_length, 2])
    try:
        for i in range(len(y)):
            y[i][0] = 1
        for original_text_words in original_text_list:
            for original_text in word_tokenize(original_text_words):
                words = word_tokenize(text)
                for i in range(len(words)):
                    if words[i].lower() == original_text.lower():
                      #  print(words[i], original_text)
                        y[i][1] = 1
                        y[i][0] = 0
    except:
        pass
    return y


In [8]:
y_list = np.zeros([len(raw_data), max_sequence_length, 2])
for body, original_text_list, i in zip(raw_data['body'], raw_data['original_text'], range(len(raw_data))):
    
    y_list[i] = generate_y(body, original_text_list, i)

In [9]:
word_tokenize(raw_data['body'].iloc[11]), raw_data['original_text'].iloc[11], y_list[11]

(['No', 'I', 'ask', 'Paytm', 'debit', 'card', 'queries'],
 ['paytm debit'],
 array([[1., 0.],
        [1., 0.],
        [1., 0.],
        [0., 1.],
        [0., 1.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.],
        [1., 0.]]))

In [10]:
ids = np.load('nearby_ids2.npy')

In [11]:
char_vocab = []
def get_character_set(text):
    for char in text:
        if not char in char_vocab:
            char_vocab.append(char)

raw_data['body'].apply(lambda x: get_character_set(x))

0        None
1        None
2        None
3        None
5        None
6        None
7        None
8        None
9        None
10       None
11       None
12       None
13       None
14       None
15       None
16       None
17       None
18       None
19       None
20       None
21       None
22       None
23       None
24       None
25       None
26       None
27       None
28       None
29       None
30       None
         ... 
21287    None
21288    None
21289    None
21290    None
21291    None
21292    None
21293    None
21294    None
21295    None
21296    None
21297    None
21298    None
21299    None
21301    None
21302    None
21304    None
21305    None
21306    None
21307    None
21309    None
21310    None
21311    None
21312    None
21313    None
21316    None
21318    None
21319    None
21320    None
21321    None
21322    None
Name: body, Length: 16623, dtype: object

In [12]:
char_vocab_len = len(char_vocab)

In [13]:
char_one_hot = np.zeros([char_vocab_len, char_vocab_len])
for i in range(len(char_vocab)):
    char_one_hot[i][i] = 1
    

In [14]:
max_char_length = 16
max_sequence_length = 20
char_dimensions = char_vocab_len
batch_size = 100
raw_data_len = len(raw_data)
ids_char = np.zeros([(raw_data_len),max_sequence_length,max_char_length ])
ids_char = ids_char + len(char_vocab) -1

In [15]:
g = 0
for i in (range((raw_data_len))):
    body = raw_data.iloc[i]['body']
    words = word_tokenize(body)
    for j in (range(len(words))):
        word = words[j]
        for k in range(len(word)):
            char = word[k]
            try:
                ids_char[i][j][k] = char_vocab.index(char)
            except:
                pass

In [16]:
ids_char.shape

(16623, 20, 16)

In [17]:
import tensorflow as tf
tf.reset_default_graph()


In [18]:
#weights = {'W_conv1': tf.get_variable("W_conv1", shape=([5,5,1,32]), dtype=tf.float32,
#              initializer=tf.random_normal_initializer()),
#           'W_conv2':                tf.get_variable("W_conv2", shape=([5,5,32,64]), dtype=tf.float32,
#              initializer=tf.random_normal_initializer()),
#           'W_fc':tf.get_variable("W_fc", shape=([(int(max_char_length//4))*(int(char_dimensions//4))*64,1024]), dtype=tf.float32,
#              initializer=tf.random_normal_initializer())
#           }



#biases = {'b_conv1':tf.get_variable("b_conv1", shape=([32]), dtype=tf.float32,
#              initializer=tf.random_normal_initializer()),
#           'b_conv2':tf.get_variable("b_conv2", shape=([64]), dtype=tf.float32,
#              initializer=tf.random_normal_initializer()),
#           'b_fc':tf.get_variable("b_fc", shape=([1024]), dtype=tf.float32,
#              initializer=tf.random_normal_initializer())
#           }
weights = {'W_conv1':tf.Variable(tf.random_normal([5,5,1,32])),
           'W_conv2':tf.Variable(tf.random_normal([5,5,32,64])),
           'W_fc':tf.Variable(tf.random_normal([(int(10//2))*(int(4//2))*32,100])),
           'out':tf.Variable(tf.random_normal([100, 10]))}

biases = {'b_conv1':tf.Variable(tf.random_normal([32])),
           'b_conv2':tf.Variable(tf.random_normal([64])),
           'b_fc':tf.Variable(tf.random_normal([100])),
           'out':tf.Variable(tf.random_normal([10]))}
n_nodes_hl1 = 40
hidden_1_layer = {'weights':tf.Variable(tf.random_normal([max_char_length * char_dimensions, n_nodes_hl1])),
                          'biases':tf.Variable(tf.random_normal([n_nodes_hl1]))}


def character_level_embedding(char_input_data):
    print('sdiugfh', char_input_data)
    char_input_data = tf.cast(char_input_data, dtype=tf.int32)
    char_one_hot_tensor = tf.convert_to_tensor(char_one_hot, dtype=tf.int32)
    char_data = tf.placeholder(shape = (max_sequence_length,max_char_length), dtype=tf.int32)
    char_data = tf.nn.embedding_lookup(char_one_hot_tensor,char_input_data)
    print('char_data', char_data)
    keep_rate = 1
    print('aya')
    
    #print(char_data.shape)
#The place holder to hold the values of new x and y at different times.
    x = tf.placeholder(dtype=tf.float32, shape=[max_sequence_length, max_char_length * char_dimensions])
    x = tf.cast(tf.reshape(char_data, shape=[max_sequence_length, max_char_length * char_dimensions]), dtype=tf.float32)
    #print(x)
#y = tf.placeholder('float', [batch_size, n_classes])

    def embedding_layer(data):
        print('aya')
        l1 = tf.add(tf.tensordot(data,hidden_1_layer['weights'], axes=1), hidden_1_layer['biases'])
        l1 = tf.nn.relu(l1)
        print(l1.shape)
        return l1

    def conv2d(x, W):
        return tf.nn.conv2d(x, W, strides=[1,1,1,1], padding='SAME')

    def maxpool2d(x):
        #                        size of window         movement of window
        return tf.nn.max_pool(x, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')


    def convolutional_neural_network(x):
        
#        tf.get_variable("W_conv1", shape=([5,5,1,32]), dtype=tf.int32,
#                      initializer=tf.ones_initializer())
#                tf.get_variable("W_conv2", shape=([5,5,32,64]), dtype=tf.int32,
#                      initializer=tf.ones_initializer())
#        
#                        tf.get_variable("W_fc", shape=([(int(max_char_length//4))*(int(char_dimensions//4))*64,1024]), dtype=tf.int32,
#                      initializer=tf.ones_initializer())

        
        

        print('og', x.shape)
        x = tf.reshape(x, shape=[max_sequence_length,10,4, 1])
        
        conv1 = tf.nn.relu(conv2d(x, weights['W_conv1']) + biases['b_conv1'])
        conv2 = maxpool2d(conv1)

        #conv2 = tf.nn.relu(conv2d(conv1, weights['W_conv2']) + biases['b_conv2'])
        #conv2 = maxpool2d(conv2)
        
        print('con2',conv2.shape)
        fc = tf.reshape(conv2,[max_sequence_length, (10//2) * (4//2) * 32])
        fc = tf.nn.relu(tf.matmul(fc, weights['W_fc'])+biases['b_fc'])
        fc = tf.nn.dropout(fc, keep_rate)
        #print(fc.shape)
        return fc

    def train_neural_network(x):    
        embedded_x = embedding_layer(x)
        print(embedded_x)
        #embedded_x = tf.reshape(x, [max_sequence_length, char_dimensions])
        convoluted_x = convolutional_neural_network(embedded_x)
        
        return convoluted_x
    
    char_data = train_neural_network(x)
    return char_data


char_input_data = tf.placeholder(dtype=tf.float32,shape= [batch_size, max_sequence_length, max_char_length])
char_embeddings = tf.map_fn(lambda x: character_level_embedding(x), char_input_data, dtype=tf.float32)



sdiugfh Tensor("map/while/TensorArrayReadV3:0", shape=(20, 16), dtype=float32)
char_data Tensor("map/while/embedding_lookup:0", shape=(20, 16, 158), dtype=int32)
aya
aya
(20, 40)
Tensor("map/while/Relu:0", shape=(20, 40), dtype=float32)
og (20, 40)
con2 (20, 5, 2, 32)


In [19]:
init_g = tf.global_variables_initializer()
init_l = tf.local_variables_initializer()
with tf.Session() as sess:
    sess.run(init_g)
    sess.run(init_l)
    p = ids_char.astype(int)[100:200]
    print(type(p[0][0][0]))
    b = sess.run([char_embeddings], feed_dict={char_input_data: p})
    print(b[0].shape)
    

<class 'numpy.int64'>
(100, 20, 100)


In [20]:
lstmUnits = 64
numClasses = 2
epochs = 10
numDimensions = 100
batchSize = 100
train_len = 14000

X_train = ids[:train_len]
y_train = y_list[:train_len]
X_test = ids[train_len:]
y_test = y_list[train_len:]
ids_char_train = ids_char[:train_len]
ids_char_test = ids_char[train_len:]




#from sklearn.model_selection import train_test_split
#X_train, X_test, y_train, y_test, ids_char_train, ids_char_test = train_test_split(ids,y_list,ids_char, test_size=.2)

len(X_train), len(y_test), len(ids_char_train)

(14000, 2623, 14000)

In [21]:
def getTrainBatch(j):
    i = j*batchSize
    arr = X_train[i:i+batchSize]
    labels = y_train[i:i+batchSize]
    arr_char = ids_char_train[i:i+batchSize]
    #print(labels[0])
    return arr, labels, arr_char

In [22]:
def getTestBatch(j):
    i = j*batchSize
    arr = X_test[i:i+batchSize]
    labels =   y_test[i:i+batchSize]
    arr_char = ids_char_test[i:i+batchSize]
    return arr, labels, arr_char

In [23]:
labels = tf.placeholder(tf.float32, [batchSize, max_sequence_length, numClasses])
input_data = tf.placeholder(tf.int32, [batchSize, max_sequence_length])

In [24]:
wordVectors = tf.convert_to_tensor(wordVectors, dtype=tf.float32)

In [25]:
data = tf.placeholder(shape = (batchSize,max_sequence_length,numDimensions), dtype=tf.float32)
data = tf.nn.embedding_lookup(wordVectors,input_data)
print(data.shape)
print(char_embeddings.shape)
data2 = tf.placeholder(shape = (batchSize,max_sequence_length,numDimensions+char_dimensions), dtype=tf.float32)
data2 = tf.concat([data, char_embeddings], axis=2)

(100, 20, 100)
(100, 20, 100)


In [26]:
fw_cell3 = tf.nn.rnn_cell.MultiRNNCell([tf.nn.rnn_cell.BasicLSTMCell(lstmUnits, state_is_tuple=True) for _ in range(2)])
bw_cell3 = tf.nn.rnn_cell.MultiRNNCell([ tf.nn.rnn_cell.BasicLSTMCell(lstmUnits, state_is_tuple=True) for _ in range(2)])

In [27]:
outputs,value2 = tf.nn.bidirectional_dynamic_rnn(fw_cell3, bw_cell3,data2,dtype=tf.float32)

In [28]:
#print(outputs)

outputs = tf.concat(outputs, 2)
#print(outputs.shape)

In [29]:
weight = tf.Variable(tf.truncated_normal([lstmUnits*2,numClasses]))
bias = tf.Variable(tf.constant(0.1, shape=[batchSize,max_sequence_length,numClasses]))

In [30]:
prediction = (tf.tensordot(outputs, weight, axes=((2,),(0,))) + bias)
prediction_arg = tf.argmax(prediction,axis=2)
label_arg = tf.argmax(labels,axis=2)
#false_positives
false_positives = tf.metrics.false_positives(predictions=prediction_arg, labels=label_arg)
false_negatives = tf.metrics.false_negatives(predictions=prediction_arg, labels=label_arg)
true_positives = tf.metrics.true_positives(predictions=prediction_arg, labels=label_arg)
true_negatives = tf.metrics.true_negatives(predictions=prediction_arg, labels=label_arg)
prec = tf.metrics.precision(predictions=prediction_arg, labels=label_arg)
rec = tf.metrics.recall(predictions=prediction_arg, labels=label_arg)
#false_positives = 1
init_g = tf.global_variables_initializer()
init_l = tf.local_variables_initializer()
with tf.Session() as sess:
    #tf.initialize_all_variables().run()
    sess.run(init_g)
    sess.run(init_l)
    nextBatch, nextBatchLabels, nextChars = getTrainBatch(2)
        #print(j/2)
    pred, pa, la, fp, fn, tp, tn = sess.run([prediction, prediction_arg, label_arg, false_positives, false_negatives, true_positives, true_negatives], {input_data: nextBatch, labels: nextBatchLabels, char_input_data: nextChars})
    print(pa.shape, la.shape, fp, fn, tp, tn)
    #a = pred[0]
    #b = nextBatchLabels
   # print(a.shape,b.shape)
    #print(a.shape)
    #print(np.argmax(a, axis=2)[0])
    #print(np.argmax(b, axis=2)[0])
    #print(np.logical_and(np.argmax(a, axis=2)[0], np.argmax(b, axis=2)[0] ))
    #print((np.equal(np.argmax(a, axis=2), np.argmax(b, axis=2)))[0])
    #print(np.mean(np.equal(np.argmax(a, axis=2), np.argmax(b, axis=2)), axis=1))
    #print(np.mean(np.equal(np.argmax(a, axis=2), np.argmax(b, axis=2)), axis=0))
    #print(np.mean(np.equal(np.argmax(a, axis=2), np.argmax(b, axis=2))))
correctPred = tf.equal(tf.argmax(prediction,axis=2), tf.argmax(labels,axis=2))
accuracy = tf.reduce_mean(tf.cast(correctPred, tf.float32))


(100, 20) (100, 20) (1271.0, 1271.0) (141.0, 141.0) (16.0, 16.0) (572.0, 572.0)


In [31]:
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=prediction, labels=labels))
optimizer = tf.train.AdamOptimizer().minimize(loss)


Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See tf.nn.softmax_cross_entropy_with_logits_v2.



In [32]:
#with tf.Session() as sess:
#    sess.run(tf.global_variables_initializer())#

#    o,w,b,p = sess.run([outputs,weight,bias,prediction],feed_dict={input_data: X_train[:100], labels: y_train[:100]})
#    print(np.array(o).shape,np.array(w).shape,np.array(b).shape,np.array(p).shape)

In [33]:
import datetime

tf.summary.scalar('Loss', loss)
tf.summary.scalar('Accuracy', accuracy)
merged = tf.summary.merge_all()
logdir = "tensorboard/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S") + "/"
writer = tf.summary.FileWriter(logdir, sess.graph)

In [34]:
sess = tf.InteractiveSession()
saver = tf.train.Saver()
init_g = tf.global_variables_initializer()
init_l = tf.local_variables_initializer()
sess.run(init_g)
sess.run(init_l)

for i in range(epochs):
    print("epoch:  "+str(i))
    #Next Batch of reviews
    losses = 0
    for j in range(len(X_train)//batchSize):
        nextBatch, nextBatchLabels, nextChars = getTrainBatch(j);
        #print(j/2)
        ll, _,pred,lab = sess.run([loss, optimizer,prediction,labels], {input_data: nextBatch, labels: nextBatchLabels, char_input_data: nextChars})
        print(ll)
        losses += ll

    #Write summary to Tensorboard
        if (j % 5 == 0):
            summary = sess.run(merged, {input_data: nextBatch, labels: nextBatchLabels, char_input_data: nextChars})
            writer.add_summary(summary, j)

        #Save the network every 10,000 training iterations
        if (j % 50 == 0 and j != 0):
            save_path = saver.save(sess, "models/pretrained_lstm.ckpt", global_step=i)
            print("saved to %s" % save_path)
    print(losses)
writer.close()

epoch:  0
0.22077945
0.2079053
0.17454389
0.13971573
0.12849453
0.1315633
0.11414803
0.12440036
0.118587054
0.11099576
0.1072645
0.10981804
0.123527005
0.1322864
0.13841373
0.14295293
0.13181126
0.1132085
0.10921219
0.09150968
0.09737611
0.09196541
0.099539325
0.08799392
0.0806541
0.096890815
0.09422344
0.07925818
0.07951962
0.08558659
0.09311615
0.08632106
0.11144341
0.1092793
0.09926801
0.09121412
0.07606805
0.08857037
0.084300004
0.077371344
0.07729346
0.06677896
0.07399739
0.08184701
0.08593546
0.08356258
0.07079471
0.073426254
0.07757882
0.061349183
0.107784025
saved to models/pretrained_lstm.ckpt-0
0.108862124
0.11754665
0.08713851
0.09493377
0.103688315
0.10630881
0.08064293
0.123865396
0.08263768
0.11945187
0.108031236
0.10362485
0.10866069
0.07171677
0.07799322
0.092167616
0.08006952
0.0841776
0.101558045
0.077029236
0.0940962
0.0836927
0.088007
0.06588414
0.06375694
0.07214543
0.07390483
0.06877568
0.05849323
0.07131946
0.070278674
0.07217214
0.07456344
0.06357213
0.066143885

0.04101217
0.05459471
0.049191687
0.05452784
0.06804218
0.06725864
0.052407604
0.062228035
0.039438773
0.05831528
0.037880927
0.04111662
0.047219787
0.05724633
0.0835381
0.069832854
0.069483764
0.073296905
0.046865035
0.05667099
0.053862352
0.052591063
0.045131523
0.052281976
7.892569240182638
epoch:  5
0.055475593
0.06051732
0.041357104
0.040551536
0.044756353
0.05066317
0.032651626
0.04297171
0.05698242
0.039838854
0.040355835
0.053466756
0.050274882
0.06054562
0.054320667
0.06306381
0.080338985
0.05857817
0.05122534
0.03715214
0.055958115
0.04705893
0.052979268
0.043031685
0.04233517
0.04260438
0.05311411
0.038414273
0.049201895
0.050634533
0.049663503
0.048134718
0.07596933
0.07091548
0.054985225
0.04995192
0.04909318
0.06764825
0.048914753
0.045591734
0.048335996
0.03874527
0.048820216
0.04245298
0.05155852
0.053885248
0.04264885
0.050497103
0.04699594
0.039298493
0.056368653
saved to models/pretrained_lstm.ckpt-5
0.06018896
0.06258185
0.051159203
0.058256194
0.052015863
0.0699303

0.046045203
0.041784573
0.067944676
0.063287735
0.06746902
0.06018025
0.05675115
0.044037905
0.0479695
0.04169403
0.052667666
0.06358591
0.03735791
0.036933966
0.049265314
0.05473896
saved to models/pretrained_lstm.ckpt-9
0.05478038
0.036486648
0.049497306
0.05435021
0.07032792
0.054071788
0.058704596
0.04399711
0.041031547
0.04328978
0.043903522
0.044675425
0.040149543
0.048533287
0.03910944
0.03230123
0.041066535
0.048391357
0.042995963
0.057948645
0.06327108
0.04636188
0.05577694
0.035797395
0.051478527
0.030253893
0.034378335
0.038465157
0.048403326
0.06843679
0.05903115
0.064792216
0.06854533
0.039507747
0.05789849
0.046830896
0.033840522
0.040746804
0.044566926
6.860182901844382


In [35]:
test_batches = 25
fin_pred = np.zeros([test_batches*100, 20, 2])
jj = 0
for j in range(test_batches):
    jj = j * 100
    nextBatch, nextBatchLabels, nextChars = getTestBatch(j);
    pred, acc, tp, tn, fp, fn, p, r = sess.run([prediction, accuracy, true_positives, true_negatives, false_positives, false_negatives, prec, rec], {input_data: nextBatch, labels: nextBatchLabels, char_input_data: nextChars})
    print(pred.shape)
    fin_pred[jj:jj+100] = pred

print(tp,tn,fp,fn,p,r)
    #    for i in range(10):

#        print(pred[i][:6])
#        print(nextBatchLabels[i][:6])
#        print('----------')

(100, 20, 2)
(100, 20, 2)
(100, 20, 2)
(100, 20, 2)
(100, 20, 2)
(100, 20, 2)
(100, 20, 2)
(100, 20, 2)
(100, 20, 2)
(100, 20, 2)
(100, 20, 2)
(100, 20, 2)
(100, 20, 2)
(100, 20, 2)
(100, 20, 2)
(100, 20, 2)
(100, 20, 2)
(100, 20, 2)
(100, 20, 2)
(100, 20, 2)
(100, 20, 2)
(100, 20, 2)
(100, 20, 2)
(100, 20, 2)
(100, 20, 2)
(4033.0, 4033.0) (44736.0, 44736.0) (801.0, 801.0) (430.0, 430.0) (0.8353474, 0.83429873) (0.9040168, 0.90365225)


In [36]:
prediction_analysis = {'original_text': [], 'pred_original_text': [], 'body': []}
#for i in range(len(y_test)):
for i in range((test_batches*100)):
    body = word_tokenize(str(raw_data[train_len:].iloc[i]['body']))
    lab_index_test = np.where(np.argmax(y_test, axis=2)[i]==1)[0]
    lab_index_pred = np.where(np.argmax(fin_pred, axis=2)[i]==1)[0]
    original_text = []
    pred_original_text = []
    for j in lab_index_test:
        original_text.append(body[j])
    for j in lab_index_pred:
        pred_original_text.append(body[j])

    prediction_analysis['original_text'].append(original_text)
    prediction_analysis['pred_original_text'].append(pred_original_text)
    prediction_analysis['body'].append(body)

In [37]:
prediction_analysis_pandas = pd.DataFrame.from_dict(prediction_analysis)

In [38]:
tru_pos = 0
fals_pos = 0 
fals_neg = 0
true_pos = []
false_pos = []
false_neg = []
def check_tp(original_text, pred_original_text):
    global tru_pos, fals_pos, fals_neg
    #print((set(original_text)))
    #print((set(pred_original_text)))
    og = set(original_text)
    pr = set(pred_original_text)
    res = og.intersection(pr)
    true_pos.append(res)
    
    false_pos.append(pr - res)
    false_neg.append(og - res)
    tru_pos += len((set(original_text).intersection(pred_original_text)))
    fals_pos += len(pr -res)
    fals_neg += len(og - res)
    
for i in range(2500):
    check_tp(prediction_analysis_pandas.iloc[i]['original_text'], prediction_analysis_pandas.iloc[i]['pred_original_text'])

In [39]:
pp = tru_pos/ (tru_pos + fals_pos)
rr = tru_pos/ (tru_pos + fals_neg)
tru_pos, fals_pos, fals_neg, pp, rr, (2*pp*rr)/(pp+rr)

(4025, 799, 411, 0.8343698175787728, 0.9073489630297565, 0.8693304535637149)

In [40]:
prediction_analysis_pandas['false_positives'] = false_pos
prediction_analysis_pandas['true_positives'] = true_pos
prediction_analysis_pandas['false_negatives'] = false_neg

In [41]:
prediction_analysis_pandas

,body,original_text,pred_original_text,false_positives,true_positives,false_negatives
0,"[i, call, on, just, dial, .., its, better, the...",[samsung],[call],{call},{},{samsung}
1,"[i, want, ambala, bus, stand, context, no, .]","[bus, stand]","[ambala, stand]",{ambala},{stand},{bus}
2,"[Beauty, parlour, for, women]","[Beauty, parlour]","[Beauty, parlour, women]",{women},"{parlour, Beauty}",{}
3,"[Its, not, a, night, club, dear]","[night, club]","[night, club]",{},"{night, club}",{}
4,"[More, Nearby, :, workshop]",[workshop],[workshop],{},{workshop},{}
5,"[Not, send, me, agai]",[agai],"[send, agai]",{send},{agai},{}
6,"[Car, workshop]","[Car, workshop]","[Car, workshop]",{},"{Car, workshop}",{}
7,[Daraganj],[Daraganj],[Daraganj],{},{Daraganj},{}
8,"[Look, for, him]","[for, him]","[Look, him]",{Look},{him},{for}
9,"[Nearby, sbi, branch]","[sbi, branch]","[sbi, branch]",{},"{sbi, branch}",{}
